# INNOV8: The Space Saga
This is the solution to the **Part 1: Decoding and Classifying Alien Commu-
nications** of the INNOV8: The Space Saga challenge. The solution is implemented in Python using TensorFlow and Scikit-learn libraries.

## 1. Importing Libraries
We first import essential libraries such as Pandas for data handling, Numpy for numerical operations, TensorFlow for building the neural network, and Scikit-learn utilities for feature extraction and preprocessing.


In [49]:
# Importing libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight

## 2. Loading and Pre-Processing Data
We load the training and test datasets, pre-process the categorical column `tail` by converting it into binary values, and apply vectorization to the text data (`message`). Additionally, numeric columns (`fingers` and `tail`) are scaled to ensure all features are on the same scale.

In [50]:
# Importing alien data and submission data
data = pd.read_csv("./data.csv")
submission = pd.read_csv("./test.csv")

# Converting the tail column into binary values
data['tail'] = data['tail'].apply(lambda x: 1 if x == 'yes' else 0)
submission['tail'] = submission['tail'].apply(lambda x: 1 if x == 'yes' else 0)

le = LabelEncoder()
scaler = StandardScaler()
vectoriser = TfidfVectorizer(ngram_range=(1, 2), min_df=2)

X_text = vectoriser.fit_transform(data['message']).toarray()
X_submission_text = vectoriser.transform(submission['message']).toarray()
X_numeric_scaled = scaler.fit_transform(data[['fingers', 'tail']].values)
X_submission_numeric_scaled = scaler.transform(submission[['fingers', 'tail']].values)

encoded_data = np.concatenate((X_text, X_numeric_scaled), axis=1)
encoded_species = le.fit_transform(data['species'])
encoded_submission = np.concatenate((X_submission_text, X_submission_numeric_scaled), axis=1)

## 3. Training the Neural Network
We define a feedforward neural network with two hidden layers, compile it using the RMSprop optimizer, and train the model on the preprocessed data. Class weights are computed to handle class imbalance, and early stopping is used to prevent overfitting.

In [51]:
X_train, y_train = encoded_data, encoded_species

class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))

model = tf.keras.Sequential([
    tf.keras.layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(len(np.unique(y_train)), activation='softmax')
])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=45, batch_size=4096, callbacks=[early_stopping], verbose=2, class_weight=class_weights_dict)

Epoch 1/45


c:\Users\ytgsg\Desktop\Dev\Innov8Hackathon\.conda\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 - 1s - 550ms/step - accuracy: 0.1000 - loss: 2.6528
Epoch 2/45
1/1 - 0s - 27ms/step - accuracy: 0.2660 - loss: 2.5745
Epoch 3/45
1/1 - 0s - 26ms/step - accuracy: 0.3540 - loss: 2.5094
Epoch 4/45
1/1 - 0s - 25ms/step - accuracy: 0.4120 - loss: 2.4525
Epoch 5/45
1/1 - 0s - 26ms/step - accuracy: 0.4220 - loss: 2.3868
Epoch 6/45
1/1 - 0s - 29ms/step - accuracy: 0.4640 - loss: 2.3080
Epoch 7/45
1/1 - 0s - 30ms/step - accuracy: 0.5060 - loss: 2.2421


c:\Users\ytgsg\Desktop\Dev\Innov8Hackathon\.conda\lib\site-packages\keras\src\callbacks\early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


Epoch 8/45
1/1 - 0s - 27ms/step - accuracy: 0.5360 - loss: 2.1727
Epoch 9/45
1/1 - 0s - 27ms/step - accuracy: 0.5720 - loss: 2.1083
Epoch 10/45
1/1 - 0s - 27ms/step - accuracy: 0.5840 - loss: 2.0352
Epoch 11/45
1/1 - 0s - 28ms/step - accuracy: 0.5680 - loss: 1.9798
Epoch 12/45
1/1 - 0s - 26ms/step - accuracy: 0.6440 - loss: 1.9013
Epoch 13/45
1/1 - 0s - 25ms/step - accuracy: 0.6400 - loss: 1.8437
Epoch 14/45
1/1 - 0s - 26ms/step - accuracy: 0.6900 - loss: 1.7706
Epoch 15/45
1/1 - 0s - 28ms/step - accuracy: 0.7400 - loss: 1.6907
Epoch 16/45
1/1 - 0s - 25ms/step - accuracy: 0.7240 - loss: 1.6336
Epoch 17/45
1/1 - 0s - 26ms/step - accuracy: 0.7320 - loss: 1.5852
Epoch 18/45
1/1 - 0s - 28ms/step - accuracy: 0.7500 - loss: 1.5154
Epoch 19/45
1/1 - 0s - 30ms/step - accuracy: 0.7940 - loss: 1.4395
Epoch 20/45
1/1 - 0s - 29ms/step - accuracy: 0.8020 - loss: 1.3705
Epoch 21/45
1/1 - 0s - 28ms/step - accuracy: 0.8260 - loss: 1.3230
Epoch 22/45
1/1 - 0s - 25ms/step - accuracy: 0.8580 - loss: 1.24

## 4. Making Predictions
Once the model is trained, we use it to make predictions on the test dataset. The predicted species are then decoded using the label encoder and saved into a CSV file for submission.


In [53]:
predictions = model.predict(encoded_submission)
le.fit(data['species'])
predicted_species = le.inverse_transform(np.argmax(predictions, axis=1))

submission['species'] = predicted_species
submission[['species']].to_csv('result.csv', index=False)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
